# Checking the result on the plaform

In [ ]:
import plotly.express as px
from scipy import stats
import numpy as np
from core.stats_utils import *
from core.pandas_utils import *
from core.config import valid_soh_points
from core.stats_utils import filter_results
from transform.fleet_info.main import fleet_info
from transform.results.main import get_results

from transform.results.processing_utils import *
from transform.results.config import *

In [ ]:
tss = get_results()

In [ ]:
# ts = tss.assign(date=lambda df: df["date"].dt.floor(UPDATE_FREQUENCY)).groupby(["vin", "date"]).agg({
#             "odometer": "last",    
#             "soh": "median",
#         }).reset_index(drop=False).pipe(filter_results, VALID_SOH_POINTS)

# ts_1= ts.pipe(filter_results, VALID_SOH_POINTS)


In [ ]:
# Application du traitement sur le DataFrame original
ts_processed = (tss
            # 3. Appliquer le traitement de SoH par véhicule
            .groupby('vin')
            .apply(process_soh_for_vehicle)
            .reset_index(level=0, drop=True)  # Supprime l'index 'vin' créé par le groupby
            
            # 1. D'abord, arrondir les dates à la semaine
            .assign(date=lambda df: df["date"].dt.floor(UPDATE_FREQUENCY))
            
            
            # 2. Grouper par vin et date pour avoir une valeur hebdomadaire
            .groupby(["vin", "date"])
            .agg({
                "odometer": "last",    
                "soh": "median",
                "model": "first",
            })
            .pipe(filter_results, VALID_SOH_POINTS)
            .reset_index()
            
            
            # 4. Reset final de l'index
            .reset_index(drop=True))

# Visualisation des résultats
fig = px.scatter(ts_processed, x="odometer", y="soh", color="model", 
                title="SoH optimisé en fonction du kilométrage")
fig.show()